In [ ]:
import polars as pl

In [ ]:
df = pl.DataFrame(
    {  # As of 14th October 2024, ~3pm UTC
        "ticker": ["AAPL", "NVDA", "MSFT", "GOOG", "AMZN"],
        "company_name": ["Apple", "NVIDIA", "Microsoft", "Alphabet (Google)", "Amazon"],
        "price": [229.9, 138.93, 420.56, 166.41, 188.4],
        "day_high": [231.31, 139.6, 424.04, 167.62, 189.83],
        "day_low": [228.6, 136.3, 417.52, 164.78, 188.44],
        "year_high": [237.23, 140.76, 468.35, 193.31, 201.2],
        "year_low": [164.08, 39.23, 324.39, 121.46, 118.35],
    }
)

print(df)

In [ ]:
# expansion in col function
eur_usd_rate = 1.09  

result = df.with_columns(
    (
        pl.col(
            "price",
            "day_high",
            "day_low",
            "year_high",
            "year_low",
        )
        / eur_usd_rate
    ).round(2)
)
print(result)

In [ ]:
# expansion by data type (float)
result = df.with_columns(
    (
        pl.col(
            pl.Float64,
            pl.Float32
        ) / eur_usd_rate
    ).round(2)
)
print(result)

In [ ]:
# expansion by pattern matching (^ start with, * wildcard, $ end with)
result = df.select(
    pl.col(
        "ticker", "^.*_high$", "^.*_low$"
    )
)
print(result)

In [ ]:
# selecting all columns
result = df.select(
    pl.all()
)
print(result)

In [ ]:
# excluding columns
result = df.select(
    pl.all().exclude("ticker")
)
print(result)

In [ ]:
# renaming with alias
result = df.select(
    pl.col("ticker").alias("ticker_symbol")
)
print(result)

In [ ]:
# prefixing and suffixing columns
gbp_usd_rate = 1.31

result = df.select(
    (pl.col("^year_.*$") / eur_usd_rate).name.prefix("in_eur_"),
    (pl.col("day_high", "day_low") / gbp_usd_rate).name.suffix("_gbp"),
)
print(result)

In [ ]:
# programmically generated expressions
def amplitude_expressions(time_periods):
    for tp in time_periods:
        yield (pl.col(f"{tp}_high") - pl.col(f"{tp}_low")).alias(f"{tp}_amplitude")


result = df.with_columns(amplitude_expressions(["day", "year"]))
print(result)